In [ ]:
import snowflake.connector

import os
 
def run_snowflake_queries():

    # Retrieve credentials from environment variables

    user = os.getenv('SNOWFLAKE_USER')

    password = os.getenv('SNOWFLAKE_PASSWORD')

    account = os.getenv('SNOWFLAKE_ACCOUNT')

    warehouse = os.getenv('SNOWFLAKE_WAREHOUSE')

    database = os.getenv('SNOWFLAKE_DATABASE')

    schema = os.getenv('SNOWFLAKE_SCHEMA')
 
    try:

        # Establish a connection to Snowflake

        conn = snowflake.connector.connect(

            user=user,

            password=password,

            account=account,

            warehouse=warehouse,

            database=database,

            schema=schema

        )
 
        # Create a cursor object to interact with the database

        cur = conn.cursor()
 
        # Set the role to ACCOUNTADMIN

        cur.execute("USE ROLE ACCOUNTADMIN")

        print("Role set to ACCOUNTADMIN")
 
        # Set the database and schema explicitly

        cur.execute(f"USE DATABASE {database}")

        cur.execute(f"USE SCHEMA {schema}")

        print(f"Using database: {database} and schema: {schema}")
 
        # Run the queries

        print("Executing queries...")
 
        # Create the table

        cur.execute("""

        CREATE OR REPLACE TABLE RAW_CO2.Daily_Measurements (

            date STRING,

            co2_ppm FLOAT

        );

        """)

        print("Table created")
 
        # Grant privileges to the ACCOUNTADMIN role

        cur.execute("""

        GRANT ALL PRIVILEGES ON TABLE RAW_CO2.Daily_Measurements TO ROLE ACCOUNTADMIN;

        """)

        print("Privileges granted")
 
        # Copy data from the stage into the table

        cur.execute("""

        COPY INTO RAW_CO2.Daily_Measurements

            FROM @RAW_CO2.CO2_EXTERNAL_STAGE

            FILE_FORMAT = (

                TYPE = 'CSV' 

                SKIP_HEADER = 1

                FIELD_OPTIONALLY_ENCLOSED_BY = '"'

            )

            ON_ERROR = 'CONTINUE';

        """)

        print("Data copied into the table")
 
        # Create a stream on the table

        cur.execute("""

        CREATE OR REPLACE STREAM RAW_CO2.DAILY_MEASUREMENTS_STREAM 

        ON TABLE RAW_CO2.Daily_Measurements;

        """)

        print("Stream created")
 
        print("Queries executed successfully!")
 
    except Exception as e:

        print(f"Error: {e}")
 
    finally:

        # Close the cursor and connection

        if 'cur' in locals():

            cur.close()

        if 'conn' in locals():

            conn.close()
 
# Call the function

if __name__ == "__main__":

    run_snowflake_queries()

 